In [1]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [2]:
df = pd.read_csv('AAPL.csv')

df['Date'] = pd.to_datetime(df['Date'])

df.index = df['Date']

df.head()


Date        Open        High         Low       Close  \
Date                                                                    
2015-01-22 2015-01-22  110.260002  112.470001  109.720001  112.400002   
2015-01-23 2015-01-23  112.300003  113.750000  111.529999  112.980003   
2015-01-26 2015-01-26  113.739998  114.360001  112.800003  113.099998   
2015-01-27 2015-01-27  112.419998  112.480003  109.029999  109.139999   
2015-01-28 2015-01-28  117.629997  118.120003  115.309998  115.309998   

             Adj Close     Volume  
Date                               
2015-01-22  103.275063   53796400  
2015-01-23  103.807991   46464800  
2015-01-26  103.918228   55615000  
2015-01-27  100.279716   95568700  
2015-01-28  105.948822  146477100

In [3]:
#fig = go.Figure(data=[go.Candlestick(x=df['Date'],
#                open=df['Open'],
#                high=df['High'],
#               low=df['Low'],
#                close=df['Close'])])

#fig.show()

In [4]:
df = df.drop(["Date"], axis=1)

In [5]:
#Exponential Moving Average  
ema = ta.EMA(df.Close,15)
df['MA15'] = ema
df = df.fillna(0)

#make MA15 into a series
MA15 = df.iloc[:,6]

# Up trend is 1, No trend is 0, and Down trend is -1
df['Trend'] = MA15.rolling('5d').apply(lambda x: np.sign(x[-1] - x[0]))

df.tail()
#If close > MA15 and MA15 is rising for last 5 days then 1 (Uptrend)
#If close < MA15 and MA15 is declining for last 5 days then -1 (Downtrend)
#If neither are true then 0 (Notrend)
def f(row):
    if row['Close'] > row['MA15'] and row['Trend'] == 1:
        val = 1
    elif row['Close'] < row['MA15'] and row['Trend'] == -1:
        val = -1
    else:
        val = 0
    return val
df['Trend'] = df.apply(f, axis=1)
df.tail()

C:\Users\JMadd\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning:

Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning



Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-14  316.700012  317.570007  312.170013  312.679993  312.679993   
2020-01-15  311.850006  315.500000  309.549988  311.339996  311.339996   
2020-01-16  313.589996  315.700012  312.089996  315.239990  315.239990   
2020-01-17  316.269989  318.739990  315.000000  318.730011  318.730011   
2020-01-21  317.190002  319.019989  316.000000  316.570007  316.570007   

              Volume        MA15  Trend  
Date                                     
2020-01-14  40488600  299.939437      1  
2020-01-15  30480900  301.364507      1  
2020-01-16  27207300  303.098942      1  
2020-01-17  34454100  305.052826      1  
2020-01-21  27664200  306.492473      1

In [53]:
Close = df.iloc[:,3]
df["UpTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5 + .5
df["DownTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5

def g(row):
    if row['Trend'] == 1:
        val = row['UpTrend']
    elif row['Trend'] == -1:
        val = row['DownTrend']
    else:
        val = 0
    return val
df['Trendi'] = df.apply(g, axis=1)
df

Open        High         Low       Close   Adj Close  \
Date                                                                     
2015-01-22  110.260002  112.470001  109.720001  112.400002  103.275063   
2015-01-23  112.300003  113.750000  111.529999  112.980003  103.807991   
2015-01-26  113.739998  114.360001  112.800003  113.099998  103.918228   
2015-01-27  112.419998  112.480003  109.029999  109.139999  100.279716   
2015-01-28  117.629997  118.120003  115.309998  115.309998  105.948822   
2015-01-29  116.320000  119.190002  115.559998  118.900002  109.247391   
2015-01-30  118.400002  120.000000  116.849998  117.160004  107.648643   
2015-02-02  118.050003  119.169998  116.080002  118.629997  108.999298   
2015-02-03  118.500000  119.089996  117.610001  118.650002  109.017654   
2015-02-04  118.500000  120.510002  118.309998  119.559998  109.853806   
2015-02-05  120.019997  120.230003  119.250000  119.940002  110.637886   
2015-02-06  120.019997  120.250000  118.449997  118.930000  109.706200   
2015-02-09  118.550003  119.839996  118.430000  119.720001  110.434944   
2015-02-10  120.169998  122.150002  120.160004  122.019997  112.556557   
2015-02-11  122.769997  124.919998  122.500000  124.879997  115.194748   
2015-02-12  126.059998  127.480003  125.570000  126.459999  116.652199   
2015-02-13  127.279999  127.279999  125.650002  127.080002  117.224121   
2015-02-17  127.489998  128.880005  126.919998  127.830002  117.915962   
2015-02-18  127.629997  128.779999  127.449997  128.720001  118.736946   
2015-02-19  128.479996  129.029999  128.330002  128.449997  118.487869   
2015-02-20  128.619995  129.500000  128.050003  129.500000  119.456444   
2015-02-23  130.020004  133.000000  129.660004  133.000000  122.684975   
2015-02-24  132.940002  133.600006  131.169998  132.169998  121.919357   
2015-02-25  131.559998  131.600006  128.149994  128.789993  118.801491   
2015-02-26  128.789993  130.869995  126.610001  130.419998  120.305084   
2015-02-27  130.000000  130.570007  128.240005  128.460007  118.497116   
2015-03-02  129.250000  130.279999  128.300003  129.089996  119.078239   
2015-03-03  128.960007  129.520004  128.089996  129.360001  119.327293   
2015-03-04  129.100006  129.559998  128.320007  128.539993  118.570885   
2015-03-05  128.580002  128.750000  125.760002  126.410004  116.606094   
...                ...         ...         ...         ...         ...   
2019-12-06  267.480011  271.000000  267.299988  270.709991  270.709991   
2019-12-09  270.000000  270.799988  264.910004  266.920013  266.920013   
2019-12-10  268.600006  270.070007  265.859985  268.480011  268.480011   
2019-12-11  268.809998  271.100006  268.500000  270.769989  270.769989   
2019-12-12  267.779999  272.559998  267.320007  271.459991  271.459991   
2019-12-13  271.459991  275.299988  270.929993  275.149994  275.149994   
2019-12-16  277.000000  280.790009  276.980011  279.859985  279.859985   
2019-12-17  279.570007  281.769989  278.799988  280.410004  280.410004   
2019-12-18  279.799988  281.899994  279.119995  279.739990  279.739990   
2019-12-19  279.500000  281.179993  278.950012  280.019989  280.019989   
2019-12-20  282.230011  282.649994  278.559998  279.440002  279.440002   
2019-12-23  280.529999  284.250000  280.369995  284.000000  284.000000   
2019-12-24  284.690002  284.890015  282.920013  284.269989  284.269989   
2019-12-26  284.820007  289.980011  284.700012  289.910004  289.910004   
2019-12-27  291.119995  293.970001  288.119995  289.799988  289.799988   
2019-12-30  289.459991  292.690002  285.220001  291.519989  291.519989   
2019-12-31  289.929993  293.679993  289.519989  293.649994  293.649994   
2020-01-02  296.239990  300.600006  295.190002  300.350006  300.350006   
2020-01-03  297.149994  300.579987  296.500000  297.429993  297.429993   
2020-01-06  293.790009  299.959991  292.750000  299.799988  299.799988   
2020-01-07  299.839996  300.899994  297.480011  298.390015  298.390015   
2020-01-08  297.160004

In [51]:
df2 = df[['Close', 'MA15', 'Trend']].copy()
df2.tail()

Close        MA15  Trend
Date                                     
2020-01-14  312.679993  299.939437      1
2020-01-15  311.339996  301.364507      1
2020-01-16  315.239990  303.098942      1
2020-01-17  318.730011  305.052826      1
2020-01-21  316.570007  306.492473      1

In [8]:
# RSI
RSI = ta.RSI(df.Close,14)

df["RSI"] = (RSI - RSI.min())/(RSI.max() - RSI.min())

#df["RSI"] = (RSI - RSI.rolling('1258 D').min())/(RSI.rolling('1258 D').max() - RSI.rolling('1258 D').min())

df.tail()

Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-14  316.700012  317.570007  312.170013  312.679993  312.679993   
2020-01-15  311.850006  315.500000  309.549988  311.339996  311.339996   
2020-01-16  313.589996  315.700012  312.089996  315.239990  315.239990   
2020-01-17  316.269989  318.739990  315.000000  318.730011  318.730011   
2020-01-21  317.190002  319.019989  316.000000  316.570007  316.570007   

              Volume        MA15  Trend  UpTrend  DownTrend  Trendi       RSI  
Date                                                                           
2020-01-14  40488600  299.939437      1      0.5        0.0     0.5  0.807358  
2020-01-15  30480900  301.364507      1      0.5        0.0     0.5  0.771804  
2020-01-16  27207300  303.098942      1      1.0        0.5     1.0  0.805234  
2020-01-17  34454100  305.052826      1      1.0        0.5     1.0  0.832147  
2020-01-21  27664200  306.492473      1      NaN        NaN     NaN  0.774140

In [11]:
# Stochastic K
df["Stoch_K"], df["Stoch_D"] = ta.STOCH(df["High"], df["Low"], df["Close"], fastk_period=14, slowk_period=3, slowk_matype=0, 
                                       slowd_period=3, slowd_matype=0)

Stoch_K = df["Stoch_K"]

df["Stoch_K"] = (Stoch_K - Stoch_K.min())/(Stoch_K.max() - Stoch_K.min())


Stock_D = df["Stock_D"]

df["Stock_D"] = (Stock_D - Stock_D.min())/(Stock_D.max() - Stock_D.min())

In [12]:
# MACD
df['MACD'], df['MACDSignal'], df['MACDHist'] = ta.MACD(df.Close, fastperiod= 12, slowperiod= 26, signalperiod= 9)

MACD = df['MACD']
df['MACD'] = (MACD - MACD.min())/(MACD.max() - MACD.min())

MACDSignal = df['MACDSignal']
df['MACDSignal'] = (MACDSignal - MACDSignal.min())/(MACDSignal.max() - MACDSignal.min())

MACDHist = df['MACDHist']
df['MACDHist'] = (MACDHist - MACDHist.min())/(MACDHist.max() - MACDHist.min())

In [13]:
# Williams R%
WILLR = ta.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14) 

df["WILLR"] = (WILLR - WILLR.min())/(WILLR.max() - WILLR.min())
df

Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-14  316.700012  317.570007  312.170013  312.679993  312.679993   
2020-01-15  311.850006  315.500000  309.549988  311.339996  311.339996   
2020-01-16  313.589996  315.700012  312.089996  315.239990  315.239990   
2020-01-17  316.269989  318.739990  315.000000  318.730011  318.730011   
2020-01-21  317.190002  319.019989  316.000000  316.570007  316.570007   

              Volume        MA15  Trend  UpTrend  DownTrend  Trendi       RSI  \
Date                                                                            
2020-01-14  40488600  299.939437      1      0.5        0.0     0.5  0.807358   
2020-01-15  30480900  301.364507      1      0.5        0.0     0.5  0.771804   
2020-01-16  27207300  303.098942      1      1.0        0.5     1.0  0.805234   
2020-01-17  34454100  305.052826      1      1.0        0.5     1.0  0.832147   
2020-01-21  27664200  306.492473      1      NaN        NaN     NaN  0.774140   

             Stoch_K   Stock_D      MACD  MACDSignal  MACDHist    Stoch_D  \
Date                                                                        
2020-01-14  0.936689  0.964763  0.994538    0.958929  0.741027  94.984096   
2020-01-15  0.895790  0.942332  0.987911    0.971539  0.671375  92.854598   
2020-01-16  0.872445  0.906521  0.990925    0.982295  0.645510  89.454770   
2020-01-17  0.920072  0.900770  1.000000    0.992914  0.643594  88.908861   
2020-01-21  0.956085  0.921636  0.993650    1.000000  0.594287  90.889810   

               WILLR  
Date                  
2020-01-14  0.858874  
2020-01-15  0.810465  
2020-01-16  0.927975  
2020-01-17  0.999702  
2020-01-21  0.916950

In [31]:
df3 = df3.fillna(0)
#df.head()

In [41]:

df3 = df[['RSI', 'Stoch_K', 'Stoch_D', 'MACD', 'WILLR', 'Trend']].copy()

#df3 = df3.reset_index()

#df3 = df3.drop(['Date'], axis = 1)

df3 = df3.fillna(0)

df3.tail()

RSI   Stoch_K    Stoch_D      MACD     WILLR  Trend
Date                                                                
2020-01-14  0.807358  0.936689  94.984096  0.994538  0.858874      1
2020-01-15  0.771804  0.895790  92.854598  0.987911  0.810465      1
2020-01-16  0.805234  0.872445  89.454770  0.990925  0.927975      1
2020-01-17  0.832147  0.920072  88.908861  1.000000  0.999702      1
2020-01-21  0.774140  0.956085  90.889810  0.993650  0.916950      1

In [42]:
input_elm = df3[['RSI', 'Stoch_K', 'Stoch_D', 'MACD', 'WILLR', 'Trend']].as_matrix()
input_elm

C:\Users\JMadd\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       ...,
       [ 0.80523368,  0.87244521, 89.4547703 ,  0.9909246 ,  0.92797476,
         1.        ],
       [ 0.83214672,  0.9200725 , 88.90886093,  1.        ,  0.9997023 ,
         1.        ],
       [ 0.7741397 ,  0.95608498, 90.88980973,  0.99365011,  0.91694976,
         1.        ]])

In [43]:
X = input_elm[:,0:5]
Y = input_elm[:,5]


In [44]:
from sklearn.model_selection import train_test_split

In [46]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(input_elm, Y, test_size=0.3)

In [47]:
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [48]:
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(880, 6) (189, 6) (189, 6) (880,) (189,) (189,)
